In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import json
import yaml
from langdetect import detect
import requests

In [3]:
def get_wikipedia(lang, name):
    # https://stackoverflow.com/a/28401782
    response = requests.get(
        f"https://{lang}.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles={name}"
    )
    pages = response.json()["query"]["pages"]
    if "-1" in pages.keys():
        return None
    else:
        for _, v in pages.items():
            return v

In [4]:
df = pd.read_csv("./对照表 2.csv")

In [5]:
df = df.fillna("")

In [6]:
df.head()

,原文,中译,属性,人物与另一词条相等情况,人物所属另一词条（团体）,作品所属另一词条（作者）,作品所属另一词条（专辑）（单曲或章节等情况下）,作品年份,mimida,翻译来源及注释（mimida）,Tian Tang,翻译来源及注释（Tian Tang）,redhousepainter,翻译来源及注释（redhousepainter）,Lil Bag,翻译来源及注释（Lil Bag）
0,《It’s Yours》,这是你的,歌,,,T La Rock,,1984,这是你的,或“属于你”,,这是你的,,,是你的,
1,Rick Rubin,里克·鲁宾,乐人,,,,,,瑞克·鲁宾,,,,,,里克·鲁宾,
2,Wayne Shorter,韦恩·肖特,乐人,,,,,,韦恩·肖特,,,,,,,
3,Neil Diamond,尼尔·戴蒙德,乐人,,,,,,尼尔·戴蒙德,,,,,,,
4,Talib Kweli,塔利卜·夸利,乐人,,,,,,,泰利布·奎力,,,,,塔利卜·夸利,更符合原始发音，而且塔利卜看着就很阿拉伯


In [7]:
data = []
for _, entry in df.iterrows():
    # handle empty row
    if not entry["原文"]:
        continue
    datum = {}
    datum["id"] = entry["原文"].replace("《", "").replace("》", "").lower()
    datum["name"] = entry["原文"].replace("《", "").replace("》", "")
    datum["consensus_translation"] = entry["中译"]
    datum["category"] = entry["属性"]
    datum["alias"] = entry["人物与另一词条相等情况"]
    datum["group"] = entry["人物所属另一词条（团体）"]
    datum["author"] = entry["作品所属另一词条（作者）"]
    datum["album"] = entry["作品所属另一词条（专辑）（单曲或章节等情况下）"]
    datum["date"] = entry["作品年份"]
    # the hacky way kind of fit for our datasets...
    datum["language"] = (
        "jp" if detect(datum["name"])[:2] in ["zh", "jp", "ko"] else "en"
    )
    datum["wikipedia"] = get_wikipedia(datum["language"], datum["name"])

    opinions = []
    for user in ["mimida", "Tian Tang", "redhousepainter", "Lil Bag"]:
        opinion = {}
        if entry[user]:
            opinion["user_name"] = user
            opinion["github_handle"] = "idmimida" if user == "mimida" else user.replace(" ", "-")
            opinion["translation"] = entry[user]
            opinion["details"] = entry[f"翻译来源及注释（{user}）"]
            opinions.append(opinion)

    datum["opinions"] = opinions

    data.append(datum)

In [8]:
def save_to_json(result):
    with open(f"../entries/{result['id']}.json", "w+") as fp:
        json.dump(result, fp, ensure_ascii=False)
    return

In [9]:
def save_to_yaml(result):
    with open(f"../entries/{result['id']}.yaml", "w+") as fp:
        yaml.dump(result, fp, allow_unicode=True)
    return

In [10]:
for d in data:
    save_to_json(d)
    # save_to_yaml(d)